In [ ]:
import pandas as pd
from glob import glob
from random import choice, sample
import altair as alt
from os.path import exists


In [ ]:
country_filepaths = glob("*/*/*/*_data.csv")
print(country_filepaths)

## Testing With One Country

In [ ]:
poland = pd.read_csv(country_filepaths[0])

In [ ]:
poland.sample(10)

In [ ]:
poland.info()

In [ ]:
poland_long = poland.melt(
        id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'],
        var_name='Year', value_name='Value'
    )

In [ ]:
poland_long[poland_long["Value"].notna()].head().max()

In [ ]:
pivoted_df = poland_long.pivot_table(
    index=['Country Name', 'Country Code', 'Year'],
    columns='Indicator Name',
    values='Value'
)

## Combining All Countries

Data is shaped as follows:
> [Country, Country Code, Year, [All Indicators]]

In [ ]:
df_list = []

In [ ]:
for country in country_filepaths:
    df = pd.read_csv(country)
    df["Indicator Name"].replace(":", "", inplace=True)
    df_long = df.melt(
        id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'],
        var_name='Year', value_name='Value'
    )
    pivoted_df = df_long.pivot_table(
    index=['Country Name', 'Country Code', 'Year'],
    columns='Indicator Name',
    values='Value'
    ).reset_index()
    df_list.append(pivoted_df)

In [ ]:
combined_data = pd.concat(df_list).reset_index()
combined_data.drop("index", axis=1, inplace=True)

In [ ]:
combined_data["Country Name"].unique()

In [ ]:
combined_data.dropna(subset="Year", inplace=True)
combined_data = combined_data.astype({"Year": int})

In [ ]:
for col in combined_data.columns:
    if combined_data[col].dtype != float:
        print(col, combined_data[col].dtype)

In [ ]:
cols = ['Country Name', 'Country Code', 'Year'] + [col for col in combined_data if col not in ['Country Name', 'Country Code', 'Year', "Indicator Name"]]
combined_data = combined_data[cols].reset_index()
combined_data.info()

In [ ]:
combined_data.sample(10)

In [ ]:
for col in [col for col in combined_data.columns if col not in ['Country Name', 'Country Code', 'Year', "Indicator Name"]]:
    if combined_data[col].nunique() < 4:
        print(col, combined_data[col].nunique(), "\n", combined_data[col].describe(), "\n")

In [ ]:

for col in [col for col in combined_data.columns if col not in ['Country Name', 'Country Code', 'Year', "Indicator Name"]]:
    if combined_data[col].count() < 10:
        print(col, combined_data[col].count(), "\n", combined_data[col].describe(), "\n", combined_data[col].isna().sum(),"\n")

In [ ]:
combined_data.columns = combined_data.columns.str.replace(':', '', regex=False)

In [ ]:
combined_data.dropna(thresh=360, inplace=True)

In [ ]:
combined_data.drop("index", axis=1, inplace=True)
combined_data.columns

In [ ]:
combined_data.to_csv("./data/world_bank/combined_world_data.csv", index=False)

## Choosing a random Indicator to Check For 3 Random Countries

In [ ]:
randomly_chosen_countries = sample(combined_data['Country Name'].unique().tolist(), 5)
filtered_data = combined_data[combined_data['Country Name'].isin(randomly_chosen_countries)]
randomly_chosen_countries

In [ ]:
randomly_chosen_column = choice([col for col in combined_data if col not in ['Country Name', 'Country Code', 'Year', "Indicator Name"]])
randomly_chosen_column 

In [ ]:
chart = alt.Chart(filtered_data).mark_line().encode(
    x='Year:O',
    y=alt.Y(randomly_chosen_column.replace(":", r"\:"), title=f'{randomly_chosen_column}'),
    color="Country Name"
).properties(
    title=f'{randomly_chosen_column} by Year'
)
chart.show()